In [7]:
# pip install --upgrade mplfinance
# pip install yfinance
# pip install PyPortfolioOpt

In [ ]:
import pandas
import numpy
import mplfinance as mpf
import yfinance as yf
from pypfopt import EfficientFrontier, risk_models, expected_returns
from matplotlib.widgets import Dropdown

In [ ]:
user_data = some_csv_file_to_pd
user_stocks = user_data['Stock'].tolist()

In [ ]:
# historical data for specific ticker

def get_historical_data(ticker):
    stock_data = yf.download(ticker, start="2024-04-16", end="2025-04-16")
    return stock_data

# graph the data
curr_stock = get_historical_data(drop_down_value)
mpf.plot(curr_stock, type="candle", volume=False, show_nontrading=True, 
         title=f"{drop_down_value} Stock Price", style="yahoo")

# dropdown
dropdown = Dropdown(plt.axes([0.3, 0.91, 0.15, 0.05]), label='Stock', options=user_stocks, value=user_stocks[0])

# update graph
def update(ticker):
    new_data = stock_data[ticker]
    fig.clf()
    mpf.plot(curr_stock, type="candle", volume=False, show_nontrading=True, 
         title=f"{drop_down_value} Stock Price", style="yahoo")

dropdown.on_changed(update)

plt.show()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2024-04-16,168.583984,172.943390,167.479200,170.942841,73711200
2024-04-17,167.210464,169.848004,167.210464,168.812899,50901200
2024-04-18,166.254944,167.847430,165.767256,167.240297,43122900
2024-04-19,164.224564,165.617978,163.308889,165.428884,67772100
2024-04-22,165.060593,166.473917,163.995629,164.742104,48116400
...,...,...,...,...,...
2025-04-09,198.850006,200.610001,171.889999,171.949997,184395900
2025-04-10,190.419998,194.779999,183.000000,189.070007,121880000


In [ ]:
# stock efficiency 
data = yf.download(user_stocks, period="1y")["Adj Close"]
returns = expected_returns.mean_historical_return(data)
cov_matrix = risk_models.sample_cov(data)

ef = EfficientFrontier(returns, cov_matrix)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()